# Interactive OFDM Symbol Builder
This notebook allows you to explore how OFDM symbols are generated from bitstreams.
You can adjust parameters like subcarrier spacing, modulation type, and simulate MIMO and multiple OFDM symbols.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider, Dropdown, Checkbox
from numpy.fft import ifft

def modulate(bits, mod_type='QPSK'):
    if mod_type == 'BPSK':
        return (1 - 2*np.array(bits)) + 0j
    elif mod_type == 'QPSK':
        symbols = []
        for i in range(0, len(bits), 2):
            b0, b1 = bits[i], bits[i+1]
            symbol = (1 - 2*b0) + 1j*(1 - 2*b1)
            symbols.append(symbol / np.sqrt(2))
        return np.array(symbols)
    elif mod_type == '16QAM':
        mapping = {
            (0,0,0,0): -3-3j, (0,0,0,1): -3-1j, (0,0,1,0): -3+3j, (0,0,1,1): -3+1j,
            (0,1,0,0): -1-3j, (0,1,0,1): -1-1j, (0,1,1,0): -1+3j, (0,1,1,1): -1+1j,
            (1,0,0,0):  3-3j, (1,0,0,1):  3-1j, (1,0,1,0):  3+3j, (1,0,1,1):  3+1j,
            (1,1,0,0):  1-3j, (1,1,0,1):  1-1j, (1,1,1,0):  1+3j, (1,1,1,1):  1+1j,
        }
        symbols = []
        for i in range(0, len(bits), 4):
            group = tuple(bits[i:i+4])
            symbol = mapping.get(group, 0)
            symbols.append(symbol / np.sqrt(10))
        return np.array(symbols)
    else:
        raise ValueError("Unsupported modulation type")

def ofdm_symbol_builder(num_bits=48, fft_size=64, cp_length=16, mod_type='QPSK', num_symbols=1, mimo=False):
    bits = np.random.randint(0, 2, num_bits)
    symbols = modulate(bits, mod_type)
    
    tx_stream = []
    for _ in range(num_symbols):
        grid = np.zeros(fft_size, dtype=complex)
        start = fft_size//2 - len(symbols)//2
        max_symbols = min(len(symbols), fft_size)
        grid[start:start + max_symbols] = symbols[:max_symbols]
        ofdm = ifft(grid)
        with_cp = np.concatenate([ofdm[-cp_length:], ofdm])
        tx_stream.extend(with_cp)
    
    if mimo:
        tx_stream = np.stack([tx_stream, tx_stream + np.random.normal(0, 0.05, len(tx_stream))])
    
    fig, axs = plt.subplots(4, 1, figsize=(10, 8))
    axs[0].stem(bits[:64])
    axs[0].set_title("Input Bits (first 64 shown)")
    axs[1].plot(np.real(symbols), np.imag(symbols), 'o')
    axs[1].set_title("Modulated Symbols")
    axs[1].set_xlabel("I")
    axs[1].set_ylabel("Q")
    axs[1].grid(True)
    axs[2].plot(np.abs(grid))
    axs[2].set_title("Frequency-Domain Grid Magnitude")
    if mimo:
        axs[3].plot(np.real(tx_stream[0]), label='Antenna 1')
        axs[3].plot(np.real(tx_stream[1]), label='Antenna 2')
        axs[3].legend()
    else:
        axs[3].plot(np.real(tx_stream))
    axs[3].set_title("Time-Domain OFDM Symbol(s) with Cyclic Prefix")
    axs[3].set_xlabel("Sample Index")
    axs[3].set_ylabel("Amplitude")
    plt.tight_layout()
    plt.show()

interact(
    ofdm_symbol_builder,
    num_bits=IntSlider(48, 24, 256, step=4),
    fft_size=IntSlider(64, 32, 256, step=32),
    cp_length=IntSlider(16, 0, 64, step=4),
    mod_type=Dropdown(options=['BPSK', 'QPSK', '16QAM'], value='QPSK', description='Modulation'),
    num_symbols=IntSlider(1, 1, 10),
    mimo=Checkbox(value=False, description='MIMO (2x1)')
)

interactive(children=(IntSlider(value=48, description='num_bits', max=256, min=24, step=4), IntSlider(value=64…

<function __main__.ofdm_symbol_builder(num_bits=48, fft_size=64, cp_length=16, mod_type='QPSK', num_symbols=1, mimo=False)>